在这个notebook，我们会尝试zhipu的模型
```bash
pip install langchain langchainhub httpx_sse

pip install zhipuai
```

In [2]:
from dotenv import load_dotenv
dot_env_path = "/Users/guochaoqun/Documents/code/.env"
load_dotenv(dotenv_path=dot_env_path)

True

采用zhipu的SDK直接调用 图像理解模型

In [3]:
import os
zhipu_api_key = os.environ.get("ZHIPU_KEY", None)
import base64
from zhipuai import ZhipuAI


In [ ]:

img_path = "/Users/guochaoqun/Downloads/test1.jpg"
with open(img_path, 'rb') as img_file:
    img_base = base64.b64encode(img_file.read()).decode('utf-8')

client = ZhipuAI(api_key=zhipu_api_key) # 填写您自己的APIKey
response = client.chat.completions.create(
    model="glm-4v",  # 填写需要调用的模型名称
    messages=[
      {
        "role": "user",
        "content": [
          {
            "type": "image_url",
            "image_url": {
                "url": img_base
            }
          },
          {
            "type": "text",
            "text": "只识别学生写入的字（包括数字和标点），并输出"
          }
        ]
      }
    ]
)
print(response.choices[0].message)


采用langchain的chatmodel格式调用
zhipu采用openai的格式

可以字迹识别的模型 能力差别: 
1. GLM-4V-FAST 不能区分 学生字迹 和 纸上的印刷字
2. GLM-4V-Plus-011 可以区分 学生字迹 和 纸上的印刷字， 但是不能按照想要的 Pydantic格式输出
3. GLM-4V-Plus-0111 可以区分 学生字迹 和 纸上的印刷字， 并且可以按照想要的 Pydantic格式输出 （6.2s 识别时长过长，需要查看其他替代模型）

In [11]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.runnables import RunnableLambda
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel
from pydantic import Field

class ModelConfig:
    model_name = "GLM-4V-Plus-0111"
    api_key = os.environ.get("ZHIPU_KEY", None)
    api_base = "https://open.bigmodel.cn/api/paas/v4/"
    temperature = 0.0

def load_img_url(img_path):
    with open(img_path, 'rb') as img_file:
        img_base = base64.b64encode(img_file.read()).decode('utf-8')
    return {"img_url": img_base}

llm = ChatOpenAI(
    temperature=ModelConfig.temperature,
    model=ModelConfig.model_name,
    openai_api_key=ModelConfig.api_key,
    openai_api_base=ModelConfig.api_base
)

class Output(BaseModel):
    text: str = Field(description="输出文字")
    handwriting_quality: int = Field(description="字迹质量。范围0-2 0分表示字迹潦草，1分表示部分字迹潦草，2分表示字迹工整")

role_prompt = SystemMessagePromptTemplate.from_template("你是一个专业的图像转文本专家，请根据图像内容，给出其中的文字。")
# img_url = load_img_url(img_path)
image_message = [{
        "type": "image_url",
        "image_url": {
            "url": '{img_url}'
        },
}]
text_message = ["只识别学生写入的字（包括数字和标点），并输出文字. 同时给出字迹质量，范围0-2 0分表示字迹潦草，1分表示部分字迹潦草，2分表示字迹工整。请按json格式输出"]
human_prompt = HumanMessagePromptTemplate.from_template(image_message + text_message)

prompt = role_prompt + human_prompt
prompt_lambda = RunnableLambda(load_img_url)
parser = PydanticOutputParser(pydantic_object=Output)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())
chain = prompt_lambda | prompt | llm | parser

result = chain.invoke( "/Users/guochaoqun/Downloads/test1.jpg")